In [1]:
import spacy

/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [3]:
nlp = spacy.load('en_core_web_md')

/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/soboroff/fb-careers/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
import json
fp = open('jobs.jl', 'rb')
jobs = [json.loads(foo) for foo in fp]

In [5]:
jobs[0]

{'url': 'https://www.facebook.com/careers/jobs/a0I1H00000LCeXGUA1/',
 'title': 'Product Management Lead, Account Integrity (Community Integrity)',
 'location': 'London, United Kingdom',
 'description': " Facebook's mission is to give people the power to build community and bring the world closer together. Through our family of apps and services, we're building a different kind of company that connects billions of people around the world, gives them ways to share what matters most to them, and helps bring people closer together. Whether we're creating new products or helping a small business expand its reach, people at Facebook are builders at heart. Our global teams are constantly iterating, solving problems, and working together to empower people around the world to build community and connect in meaningful ways. Together, we can help people build stronger communities — we're just getting started.  Protecting the safety and integrity of the Facebook community is the company's top prio

In [7]:
parsed = [nlp(job['description']) for job in jobs]

In [8]:
parsed[0]

 Facebook's mission is to give people the power to build community and bring the world closer together. Through our family of apps and services, we're building a different kind of company that connects billions of people around the world, gives them ways to share what matters most to them, and helps bring people closer together. Whether we're creating new products or helping a small business expand its reach, people at Facebook are builders at heart. Our global teams are constantly iterating, solving problems, and working together to empower people around the world to build community and connect in meaningful ways. Together, we can help people build stronger communities — we're just getting started.  Protecting the safety and integrity of the Facebook community is the company's top priority and we’re looking for an experienced PM leader to help direct the Account Integrity feature teams as we accelerate towards the next billion users globally. Account Integrity’s mission is reducing ne

In [11]:
for ent in parsed[0].ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Facebook 1 9 PERSON
billions 202 210 CARDINAL
Facebook 423 431 ORG
Facebook 753 761 PERSON
2B+ 1057 1060 GPE
monthly 1061 1068 DATE
Facebook 1172 1180 ORG
… 1678 1679 ORG
Salary 1917 1923 PERSON
Laundry Benefit 2103 2118 PERSON
21st March 2018 2128 2143 DATE
18th April 2018 2158 2173 DATE
Product Management 2417 2435 ORG
8+ years 2466 2474 DATE
3+ 2506 2508 DATE
years 2509 2514 DATE
– 2903 2904 DATE


In [12]:
for np in parsed[0].noun_chunks:
    print(np)

 Facebook's mission
people
the power
community
the world
our family
apps
services
we
a different kind
company
billions
people
the world
them
ways
what
them
people
we
new products
a small business
its reach
people
Facebook
builders
heart
Our global teams
problems
people
the world
community
meaningful ways
we
people
stronger communities
we
the safety
integrity
the Facebook community
the company's top priority
we
an experienced PM leader
the Account Integrity feature teams
we
the next billion users
Account Integrity
’s mission
negative experiences
abusive accounts
over 2B+ monthly active users
an extremely impactful and rewarding opportunity
Facebook
integrity efforts
fake news
misinformation
harmful behaviours
bullying
harassment
The team
data-analysis
data-driven decisions
innovation
detection
human review systems
machine learning
user experiences
the community
fake accounts
users
unfortunate bad events
activities
–
likes
comments
a user’s account
someone
the account owner
The goals
aim

Let's do a topic model using Spacy noun chunks as features.

In [18]:
def phraser(txt): return [chunk.text for chunk in nlp(txt).noun_chunks]

In [19]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
no_features = 10000
tfidf_vectorizer = TfidfVectorizer(analyzer=phraser)
docs = [j['description'] for j in jobs]
tfidf = tfidf_vectorizer.fit_transform(docs)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print(tfidf_feature_names)

[" Facebook's mission", ' Over a billion people', '"The Studio', '"roll-up the sleeves', '#LI-DNI', '%', '(AI', '(Ads Manager', '(Computer Science', '(Established', '(FEA/Simulation experience', '(FRL', '(GDPR', '(MS/MA/MBA', '(PE', '(PHP, SQL', '(PHP, SQL) Experience', '(REL', '(TX', '(Yara, ClamAV, Suricata) Experience', '(e.g. SPICE, MATLAB) Experience', '(e.g. data integrity', '(e.g., Product', '(e.g., statistics', '(http://force.com//) business applications', '(https://research.facebook.com/', '(ie: operations', '(multi-Gigabit', '(test algorithm', '(x86, ARM', ') Advanced technical degree', ') Experience', '* Experience', ', FPGAs', ', candidate recruiting plans', ', data-driven, and compelling reports', ', fast-paced projects', ', full cycle staffing plans', ', high speed interconnect', ', natural language processing', ', program/product management', ', track record', '-  Collaborate', '- project management', '-Application security concepts', '-Camera module mechanical design an

In [21]:
from sklearn.decomposition import NMF
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
people, we, the world, Facebook, them, community, We, problems, experience, apps
Topic 1:
people, measurement, the impact, workflows, data analysis, a variety, our platform, businesses, we, regular updates
Topic 2:
people, projects, we, business value Build interfaces, managing resources, re-imagining productivity, system stability, Technical ownership, change management policies, production quality software Experience
Topic 3:
our team, We, people, end, creative sourcing techniques, internal stakeholders, key cross-functional stakeholders, meaningful relationships, pipeline activity, other areas
Topic 4:
data-informed initiatives, insights, the quality, our platform, businesses, Engineering, your positions, impact metrics, new strategies, tactics
Topic 5:
startups, developers, partnerships, direct and active relationships, business development, the highest personal integrity, the time, PR, an internal advocate, product
Topic 6:
scale, technical and non-technical audiences Exp